In [1]:
import nltk
from pprint import pprint as print
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath
from gensim.models.word2vec import Word2Vec, Text8Corpus
import numpy as np
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Prime prove fastText
il modello creato è stato creato con 1 epoca dal modello wikipedia, da migliorare e diminuire le parole, 
introducendo un dizionario di paroli comuni italiane (vedi come fa il professore)

le funzioni seguenti servono per fare lo spell checker e non per predirre la parola sucessiva

In [2]:
model = FT_gensim.load('WikiFast1tris.model') #modello creato con fast text 1 epoca (da migliorare)

2020-04-02 19:14:33,938 : INFO : loading FastText object from WikiFast1tris.model
2020-04-02 19:14:34,117 : INFO : loading wv recursively from WikiFast1tris.model.wv.* with mmap=None
2020-04-02 19:14:34,118 : INFO : loading vectors from WikiFast1tris.model.wv.vectors.npy with mmap=None
2020-04-02 19:14:34,200 : INFO : loading vectors_vocab from WikiFast1tris.model.wv.vectors_vocab.npy with mmap=None
2020-04-02 19:14:34,272 : INFO : loading vectors_ngrams from WikiFast1tris.model.wv.vectors_ngrams.npy with mmap=None
2020-04-02 19:14:36,332 : INFO : setting ignored attribute vectors_norm to None
2020-04-02 19:14:36,334 : INFO : setting ignored attribute vectors_vocab_norm to None
2020-04-02 19:14:36,334 : INFO : setting ignored attribute vectors_ngrams_norm to None
2020-04-02 19:14:36,334 : INFO : setting ignored attribute buckets_word to None
2020-04-02 19:14:36,335 : INFO : loading vocabulary recursively from WikiFast1tris.model.vocabulary.* with mmap=None
2020-04-02 19:14:36,335 : INF

In [3]:
path = '/Users/stefanoraimondousai/Documents/ReadingCourse/text_pulito.txt' #testo con chat marta
sentences = datapath(path)

In [4]:
model.train(sentences,epochs=5,total_examples=model.corpus_count, compute_loss=False)

2020-04-02 19:14:40,585 : INFO : training model with 8 workers on 63937 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2020-04-02 19:14:40,595 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-04-02 19:14:40,599 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-04-02 19:14:40,599 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-04-02 19:14:40,600 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-04-02 19:14:40,602 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-04-02 19:14:40,602 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-04-02 19:14:40,603 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-04-02 19:14:40,604 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-04-02 19:14:40,605 : INFO : EPOCH - 1 : training on 66 raw words (45 effective words) took 0.0s, 3414 effective words

In [5]:
dictionary = list(model.wv.vocab.keys())

In [6]:
    def predict_output_word(model, context_words_list, topn=10):
        from numpy import exp, dot, zeros, random, dtype, float32 as REAL,\
        uint32, seterr, array, uint8, vstack, fromstring, sqrt,\
        empty, sum as np_sum, ones, logaddexp, log, outer
        from scipy.special import expit
        from gensim import utils, matutils  # utility fnc for pickling, common scipy operations etc

        if not model.negative:
            raise RuntimeError(
                "We have currently only implemented predict_output_word for the negative sampling scheme, "
                "so you need to have run word2vec with negative > 0 for this to work."
            )

        if not hasattr(model.wv, 'vectors') or not hasattr(model.trainables, 'syn1neg'):
            raise RuntimeError("Parameters required for predicting the output words not found.")

        word_vocabs = [model.wv.vocab[w] for w in context_words_list if w in model.wv.vocab]
        if not word_vocabs:
            warnings.warn("All the input context words are out-of-vocabulary for the current model.")
            return None

        word2_indices = [word.index for word in word_vocabs]

        l1 = np_sum(model.wv.vectors[word2_indices], axis=0)
        if word2_indices and model.cbow_mean:
            l1 /= len(word2_indices)

        # propagate hidden -> output and take softmax to get probabilities
        prob_values = exp(dot(l1, model.trainables.syn1neg.T))
        prob_values /= sum(prob_values)
        top_indices = matutils.argsort(prob_values, topn=topn, reverse=True)
        # returning the most probable output words with their probabilities
        return [(model.wv.index2word[index1], prob_values[index1]) for index1 in top_indices]

In [7]:
def weight(word, similar_word, similarity, constant = 0.5, min_weight=3):
    min_similarity = similarity > constant   
    word_len = 1 if len(word) <= 4 else 2 #se è minore di 4 è un articolo quasi sicuramente
    exist = 1 if word in dictionary else 0 # esiste nel dizionario?
    min_len = len(similar_word) > 4 #se è più lungo di 4 evito che siano articoli
    check_first_word = 1 if word[0] == similar_word[0] else 0 #controllo la prima lettera della parola
    levenshtein_distance = nltk.edit_distance(word, similar_word) <= word_len #sostituire con quello di marta
    weight = word_len + min_similarity + min_len + exist + levenshtein_distance
    if weight > min_weight:
        return True
    else:
        return False
    

In [8]:
def spellChecker(word):
    most_similar = model.wv.most_similar(word, topn=50) #parole simili dal modello
    result = list(filter(lambda x: weight(word, x[0], x[1]), most_similar))#filtro con la funzione weight
    result = {'prediction': result[:3]} #dizionario con le prime 5 parole
    return result

In [9]:
spellChecker('prpfessore')

2020-04-02 19:14:45,452 : INFO : precomputing L2-norms of word weight vectors
2020-04-02 19:14:45,534 : INFO : precomputing L2-norms of ngram weight vectors


{'prediction': [('professore', 0.6842303276062012),
  ('confessore', 0.6782220602035522),
  ('professoressa', 0.5594971179962158)]}

In [10]:
predict_output_word(model, ['devi', 'salire', 'in'], topn=50)

[('location', 0.0006342294),
 ('banca', 0.000510357),
 ('ricky', 0.00047233634),
 ('manager', 0.00042126453),
 ('sanpaolo', 0.00041730888),
 ('salvare', 0.0003768027),
 ('sam', 0.00036531643),
 ('ramses', 0.00036193614),
 ('lazio', 0.00035225626),
 ('quota', 0.00034832882),
 ('torna', 0.00033939298),
 ('gruppo', 0.0003303498),
 ('riesce', 0.00033010312),
 ('controller', 0.0003288965),
 ('dovrà', 0.000321276),
 ('far', 0.00031655602),
 ('fare', 0.0003151168),
 ('salvezza', 0.0003143854),
 ('posto', 0.0003118625),
 ('intesa', 0.00029188063),
 ('milioni', 0.0002914475),
 ('comincia', 0.00029035666),
 ('uscire', 0.0002885761),
 ('horowitz', 0.0002813761),
 ('fa', 0.00027646072),
 ('lester', 0.00027384903),
 ('parco', 0.0002697703),
 ('ligure', 0.00026862972),
 ('prussia', 0.00026022564),
 ('picchiata', 0.00025876067),
 ('allenatore', 0.0002582462),
 ('newman', 0.0002566271),
 ('qui', 0.0002476692),
 ('girone', 0.00024693823),
 ('creditore', 0.00023616463),
 ('portarsi', 0.00023424067),
 ('